# 🎯 Part 7.3: 时变处理效应 (Time-Varying Treatment Effects)

## 学习目标

完成本练习后，你将能够：

1. 理解时变处理的挑战和现实意义
2. 掌握边际结构模型 (Marginal Structural Models, MSM) 的核心思想
3. 实现时变处理的逆概率加权 (IPTW)
4. 理解 G-computation 方法
5. 掌握序贯忽略假设 (Sequential Ignorability)
6. 在真实业务场景中应用时变处理分析

---

## 🎬 真实场景：在线教育平台的个性化推送策略

想象你是某在线教育平台的数据科学家，负责优化用户的学习推送策略。

### 场景描述

平台有一项「学习提醒」功能，每天可以给用户发送学习推送通知：

- **👨‍💼 努力的小张**：前 3 天都认真学习，系统每天推送。第 4 天因为推送太频繁有点厌烦，学习时长下降了
- **👩‍🎓 摆烂的小李**：前 3 天都没学习，系统停止推送。第 4 天突然想学习了，但没有推送提醒错过了
- **🧓 佛系的老王**：时学时不学，系统根据他的历史行为动态调整推送频率

### 核心问题

**问题 1：** 如何评估「连续 7 天推送」vs「不推送」对用户最终学习成果的影响？

**问题 2：** 用户过去的学习行为会影响：
- 今天是否收到推送（处理分配）
- 未来的学习表现（结果变量）

这就是**时间依赖性混淆**（time-dependent confounding）！

### 为什么传统方法失效？

如果用传统的 IPW 或 PSM:

```
❌ 错误做法：只看最后一天的推送状态
   问题：忽略了历史推送的累积效应

❌ 错误做法：用历史学习时长作为协变量调整
   问题：历史学习时长既是混淆因子，又是中间结果！
   这会导致「过度调整偏差」（over-adjustment bias）
```

### 时变处理的挑战

时间维度上的因果关系链：

```
Day 1:  推送1 → 学习1 ─┐
                        ├→ 影响 → 推送2 → 学习2 ─┐
                        │                        ├→ 影响 → ...
                        └────────────────────────┘
                         (混淆因子)  (处理)  (中间结果 & 混淆因子)
```

**核心矛盾：** 要调整混淆，必须控制历史学习时长；但控制历史学习时长会阻断因果路径！

**解决方案：** 边际结构模型 (MSM) + 逆概率加权 (IPTW)

---

## 📐 核心概念

### 1. 时变处理 (Time-Varying Treatment)

#### 定义

个体在多个时间点接受处理，记为 $\bar{A}_t = (A_0, A_1, ..., A_t)$

- $A_t \in \{0, 1\}$：第 $t$ 时刻是否接受处理
- $\bar{A}_t$：从开始到时刻 $t$ 的处理历史

#### 符号说明

| 符号 | 含义 |
|------|------|
| $A_t$ | 时刻 $t$ 的处理状态 (0 或 1) |
| $\bar{A}_t$ | 处理历史 $(A_0, ..., A_t)$ |
| $L_t$ | 时刻 $t$ 的协变量（包括既往结果） |
| $\bar{L}_t$ | 协变量历史 $(L_0, ..., L_t)$ |
| $Y$ | 最终结果 |

#### 直观理解

就像问：**"如果小张在整个学习周期都收到推送，相比完全不推送，最终成绩会提升多少？"**

---

### 2. 时间依赖性混淆 (Time-Dependent Confounding)

#### 核心问题

$$L_t \text{ (协变量)} \rightarrow \begin{cases} A_{t+1} \text{ (未来处理)} \\ Y \text{ (最终结果)} \end{cases}$$

并且 $L_t$ 本身受到过去处理 $\bar{A}_{t-1}$ 的影响！

#### 生活化类比：吃药的困境

想象你在治疗一种慢性病：

1. **第 1 天**：医生根据你的症状严重程度决定是否开药
2. **第 2 天**：
   - 如果昨天吃药了 → 症状可能好转
   - 症状好转 → 医生今天可能不开药
   - 症状好转 → 你最终康复概率更高
3. **问题来了**：昨天的症状既影响今天的处理，又影响最终康复

传统方法调整「昨天的症状」会切断因果路径！

---

### 3. 序贯忽略假设 (Sequential Ignorability)

#### 数学形式

$$Y(\bar{a}) \perp A_t \mid \bar{A}_{t-1} = \bar{a}_{t-1}, \bar{L}_t$$

#### 白话翻译

**在给定历史处理和协变量的条件下，当前处理分配与潜在结果独立**

也就是说：
- 如果我们知道小张过去的学习行为和推送历史
- 今天是否推送就是「随机」的（没有其他未观测的混淆因子）

#### 与标准忽略性的区别

| 假设 | 标准忽略性 | 序贯忽略性 |
|------|-----------|------------|
| **适用** | 单时点处理 | 多时点处理 |
| **条件** | $Y(a) \perp A \mid X$ | $Y(\bar{a}) \perp A_t \mid \bar{A}_{t-1}, \bar{L}_t$ |
| **含义** | 控制基线协变量 | 控制处理和协变量的完整历史 |

---

### 4. 边际结构模型 (Marginal Structural Models, MSM)

#### 核心思想

**如果整个样本都遵循某个处理方案 $\bar{a}$，平均结果会是多少？**

$$E[Y(\bar{a})] = \beta_0 + \beta_1 \sum_{t=0}^{T} a_t + \beta_2 f(\bar{a})$$

#### 常见的 MSM 形式

**1. 累积剂量模型**

$$E[Y(\bar{a})] = \beta_0 + \beta_1 \sum_{t=0}^{T} a_t$$

直观：总推送次数的效应

**2. 最近处理模型**

$$E[Y(\bar{a})] = \beta_0 + \beta_1 a_T + \beta_2 a_{T-1} + ...$$

直观：最近的推送影响更大

**3. 处理方案对比**

$$E[Y(\bar{a})] = \beta_0 + \beta_1 \cdot I(\bar{a} = \text{"总是处理"}) + \beta_2 \cdot I(\bar{a} = \text{"从不处理"})$$

---

### 5. 时变 IPTW (Inverse Probability of Treatment Weighting)

#### 稳定化权重公式

$$SW_i = \prod_{t=0}^{T} \frac{P(A_{it} = a_{it} \mid \bar{A}_{i,t-1})}{P(A_{it} = a_{it} \mid \bar{A}_{i,t-1}, \bar{L}_{it})}$$

#### 公式拆解

- **分子**：只基于处理历史的处理概率（边际概率）
- **分母**：基于处理历史和协变量历史的处理概率（条件概率）

#### 为什么要稳定化？

- **未稳定化权重**：$W = \prod \frac{1}{P(A_t \mid \bar{A}_{t-1}, \bar{L}_t)}$
- **问题**：如果某个时点的处理概率很小 → 权重爆炸！
- **解决**：分子加上边际概率，使权重接近 1

#### 直观理解：创造伪总体

IPTW 的本质是创造一个**伪总体**，在这个总体中：

```
原始数据:  推送 ← 历史学习表现 → 最终成绩
              ↓                       ↓
          (混淆!)                (有偏!)

加权后:   推送 ⊥ 历史学习表现 → 最终成绩
              ↓                       ↓
          (随机!)               (无偏!)
```

---

### 6. G-Computation (参数化 G-Formula)

#### 核心思路

**Step 1:** 对每个时点的结果建模

$$E[Y \mid \bar{A}_t, \bar{L}_t] = g_t(\bar{a}_t, \bar{l}_t; \theta_t)$$

**Step 2:** 递归模拟干预后的潜在结果

```python
for t in 0..T:
    # 设置干预
    A_t = intervention(t)
    
    # 预测结果
    L_{t+1} = predict(A_t, L_t, history)
    
# 最终结果
Y = predict(A_bar, L_bar)
```

#### 与 MSM-IPTW 的对比

| 方法 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| **MSM-IPTW** | ✅ 非参数化<br>✅ 稳健性强 | ❌ 权重不稳定<br>❌ 需要大样本 | 处理机制简单，样本量大 |
| **G-Computation** | ✅ 效率高<br>✅ 可以处理复杂时间动态 | ❌ 模型依赖性强<br>❌ 模型误设风险高 | 有充分的领域知识建模 |

---

## 🔧 环境准备

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from typing import Tuple, List, Dict
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# Plotly 颜色方案
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'neutral': '#828282',
    'treated': '#E74C3C',
    'control': '#3498DB'
}

print("✅ 环境准备完成！")
print(f"NumPy 版本: {np.__version__}")
print(f"Pandas 版本: {pd.__version__}")

## 📊 数据生成：在线教育推送场景

### 场景设定

- **时间跨度**：连续 7 天
- **处理**：每天是否推送学习提醒 (0/1)
- **协变量**：每天的学习时长
- **结果**：第 7 天的测验成绩

### 因果机制

```
Day t:
  学习时长_t ──→ 推送决策_{t+1}
      ↓              ↓
      └──→ 影响 ──→ 学习时长_{t+1}
                      ↓
                   最终成绩
```

In [ ]:
def generate_time_varying_data(n_users: int = 1000, n_days: int = 7, seed: int = 42) -> pd.DataFrame:
    """
    生成时变处理数据：在线教育推送场景
    
    参数:
        n_users: 用户数量
        n_days: 观测天数
        seed: 随机种子
    
    返回:
        DataFrame with columns: user_id, day, push, study_hours, baseline_motivation, final_score
    """
    np.random.seed(seed)
    
    data = []
    
    for user_id in range(n_users):
        # 基线特征：用户的内在学习动机（固定）
        baseline_motivation = np.random.normal(50, 15)
        
        # 初始学习时长（基于动机）
        study_hours_prev = np.random.normal(baseline_motivation / 25, 0.5)
        study_hours_prev = np.clip(study_hours_prev, 0, 5)
        
        user_history = []
        
        for day in range(n_days):
            # Day 0: 基线，不推送
            if day == 0:
                push = 0
                study_hours = study_hours_prev
            else:
                # 推送决策：基于前一天的学习时长（时间依赖性混淆！）
                # 学习时长越短，越可能推送
                propensity = 1 / (1 + np.exp(2 * (study_hours_prev - 1.5)))
                push = np.random.binomial(1, propensity)
                
                # 今天的学习时长受到：
                # 1. 基线动机
                # 2. 前一天的学习时长（惯性）
                # 3. 是否推送（处理效应）
                
                # 基础学习时长
                base_study = 0.5 * study_hours_prev + 0.01 * baseline_motivation
                
                # 推送的效应（异质性）
                # - 低动机用户：推送效果好 (+0.5 小时)
                # - 高动机用户：推送效果一般 (+0.1 小时)
                if baseline_motivation < 50:
                    push_effect = push * 0.5
                else:
                    push_effect = push * 0.1
                
                # 累积疲劳效应（推送太多会厌烦）
                total_pushes = sum([h['push'] for h in user_history])
                fatigue = -0.05 * total_pushes if total_pushes > 3 else 0
                
                study_hours = base_study + push_effect + fatigue + np.random.normal(0, 0.3)
                study_hours = np.clip(study_hours, 0, 5)
            
            user_history.append({
                'user_id': user_id,
                'day': day,
                'push': push,
                'study_hours': study_hours,
                'baseline_motivation': baseline_motivation
            })
            
            study_hours_prev = study_hours
        
        # 最终成绩：基于整个学习轨迹
        total_study = sum([h['study_hours'] for h in user_history])
        total_pushes = sum([h['push'] for h in user_history])
        
        final_score = (
            50 +                           # 基础分
            0.3 * baseline_motivation +    # 动机影响
            3 * total_study +              # 总学习时长影响
            np.random.normal(0, 5)         # 随机噪声
        )
        final_score = np.clip(final_score, 0, 100)
        
        # 记录最终成绩
        for h in user_history:
            h['final_score'] = final_score
        
        data.extend(user_history)
    
    return pd.DataFrame(data)

# 生成数据
df = generate_time_varying_data(n_users=1000, n_days=7)

print("✅ 数据生成完成！")
print(f"\n数据形状: {df.shape}")
print(f"\n前 10 行:")
print(df.head(10))

# 汇总统计
print(f"\n=== 汇总统计 ===")
print(f"用户数: {df['user_id'].nunique()}")
print(f"观测天数: {df['day'].nunique()}")
print(f"总推送次数: {df['push'].sum()}")
print(f"推送比例: {df['push'].mean():.2%}")
print(f"\n平均每用户推送次数: {df.groupby('user_id')['push'].sum().mean():.2f}")
print(f"平均每天学习时长: {df['study_hours'].mean():.2f} 小时")
print(f"平均最终成绩: {df.groupby('user_id')['final_score'].first().mean():.2f}")

## 📈 可视化：理解时变数据结构

In [ ]:
# 可视化 1: 个体学习轨迹（选取 10 个用户）

sample_users = df['user_id'].unique()[:10]
df_sample = df[df['user_id'].isin(sample_users)]

fig = go.Figure()

for user in sample_users:
    user_data = df_sample[df_sample['user_id'] == user]
    
    # 学习时长轨迹
    fig.add_trace(go.Scatter(
        x=user_data['day'],
        y=user_data['study_hours'],
        mode='lines+markers',
        name=f'User {user}',
        hovertemplate='<b>User %{text}</b><br>Day: %{x}<br>Study Hours: %{y:.2f}<br>Push: %{customdata}',
        text=[user] * len(user_data),
        customdata=user_data['push']
    ))

fig.update_layout(
    template='plotly_white',
    title='个体学习轨迹：学习时长随时间变化',
    xaxis_title='天数',
    yaxis_title='学习时长（小时）',
    hovermode='closest',
    height=500
)

fig.show()

In [ ]:
# 可视化 2: 时间依赖性混淆

# 计算前一天学习时长对今天推送的影响
# 为每个用户创建滞后变量
df_with_lag = df.sort_values(['user_id', 'day']).copy()
df_with_lag['study_hours_lag'] = df_with_lag.groupby('user_id')['study_hours'].shift(1)

# 只保留 day > 0 的数据（因为 day 0 没有前一天）
df_lag = df_with_lag[df_with_lag['day'] > 0].dropna(subset=['study_hours_lag'])

# 分组统计
df_lag['study_bin'] = pd.cut(df_lag['study_hours_lag'], bins=5)
confounding_stats = df_lag.groupby('study_bin', observed=True).agg({
    'push': 'mean',
    'final_score': 'mean'
}).reset_index()
confounding_stats['study_bin_str'] = confounding_stats['study_bin'].astype(str)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('前一天学习时长 → 今天推送概率', '前一天学习时长 → 最终成绩')
)

# 左图：学习时长 → 推送概率
fig.add_trace(
    go.Bar(
        x=confounding_stats['study_bin_str'],
        y=confounding_stats['push'],
        marker_color=COLORS['danger'],
        name='推送概率',
        showlegend=False
    ),
    row=1, col=1
)

# 右图：学习时长 → 最终成绩
fig.add_trace(
    go.Bar(
        x=confounding_stats['study_bin_str'],
        y=confounding_stats['final_score'],
        marker_color=COLORS['success'],
        name='最终成绩',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_xaxes(title_text='前一天学习时长（小时）', row=1, col=1)
fig.update_xaxes(title_text='前一天学习时长（小时）', row=1, col=2)
fig.update_yaxes(title_text='推送概率', row=1, col=1)
fig.update_yaxes(title_text='平均最终成绩', row=1, col=2)

fig.update_layout(
    template='plotly_white',
    title_text='时间依赖性混淆：学习时长同时影响推送和结果',
    height=400
)

fig.show()

print("\n💡 观察：")
print("- 学习时长低的用户更可能收到推送（系统试图激励他们）")
print("- 学习时长低的用户最终成绩也更低")
print("- 这就是时间依赖性混淆！简单比较推送 vs 不推送会有偏差")

In [ ]:
# 可视化 3: 朴素估计的偏差

# 计算每个用户的总推送次数
user_summary = df.groupby('user_id').agg({
    'push': 'sum',
    'final_score': 'first',
    'baseline_motivation': 'first'
}).reset_index()
user_summary['push_group'] = pd.cut(user_summary['push'], bins=[0, 2, 4, 7], labels=['低频 (0-2)', '中频 (3-4)', '高频 (5-7)'])

# 分组统计
naive_stats = user_summary.groupby('push_group')['final_score'].agg(['mean', 'std', 'count']).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=naive_stats['push_group'],
    y=naive_stats['mean'],
    error_y=dict(type='data', array=naive_stats['std']),
    marker_color=COLORS['warning'],
    text=[f"{m:.1f}" for m in naive_stats['mean']],
    textposition='outside'
))

fig.update_layout(
    template='plotly_white',
    title='朴素分析：推送频率与最终成绩（有偏！）',
    xaxis_title='推送频率',
    yaxis_title='平均最终成绩',
    height=400
)

fig.show()

print("\n❌ 朴素结论（错误！）：")
print(f"   推送次数越多，成绩反而越低！")
print(f"   是不是推送有害？")
print("\n✅ 真相：")
print("   推送次数多 ← 历史学习时长低 → 最终成绩低")
print("   这是选择偏差，不是因果效应！")

## 🎯 实现：边际结构模型 + 时变 IPTW

### 实现步骤

1. **估计倾向得分**：对每个时点 $t$，估计 $P(A_t = 1 \mid \bar{A}_{t-1}, \bar{L}_t)$
2. **计算稳定化权重**：$SW = \prod_t \frac{P(A_t \mid \bar{A}_{t-1})}{P(A_t \mid \bar{A}_{t-1}, \bar{L}_t)}$
3. **拟合 MSM**：使用权重拟合边际结构模型
4. **估计因果效应**：对比不同处理方案下的预期结果

In [ ]:
# Step 1: 估计倾向得分

def estimate_propensity_scores(df: pd.DataFrame) -> pd.DataFrame:
    """
    估计每个时点的倾向得分
    
    对于每个 day > 0:
        - 分子：P(A_t | A_{t-1})
        - 分母：P(A_t | A_{t-1}, L_{t-1})
    """
    df = df.copy()
    df['ps_numerator'] = 1.0
    df['ps_denominator'] = 1.0
    
    for day in range(1, df['day'].max() + 1):
        # 当天的数据
        day_data = df[df['day'] == day].copy()
        
        # 前一天的数据
        prev_day_data = df[df['day'] == day - 1][['user_id', 'push', 'study_hours']]
        prev_day_data.columns = ['user_id', 'push_lag', 'study_hours_lag']
        
        # 合并
        day_merged = day_data.merge(prev_day_data, on='user_id')
        
        # === 分子：P(A_t | A_{t-1}) ===
        X_num = day_merged[['push_lag']].values
        y = day_merged['push'].values
        
        model_num = LogisticRegression(max_iter=1000)
        model_num.fit(X_num, y)
        ps_num = model_num.predict_proba(X_num)[:, 1]
        
        # === 分母：P(A_t | A_{t-1}, L_{t-1}) ===
        X_denom = day_merged[['push_lag', 'study_hours_lag']].values
        
        model_denom = LogisticRegression(max_iter=1000)
        model_denom.fit(X_denom, y)
        ps_denom = model_denom.predict_proba(X_denom)[:, 1]
        
        # 根据实际处理状态选择概率
        ps_num_final = np.where(y == 1, ps_num, 1 - ps_num)
        ps_denom_final = np.where(y == 1, ps_denom, 1 - ps_denom)
        
        # 更新 dataframe
        df.loc[df['day'] == day, 'ps_numerator'] = ps_num_final
        df.loc[df['day'] == day, 'ps_denominator'] = ps_denom_final
    
    return df

# 估计倾向得分
df = estimate_propensity_scores(df)

print("✅ 倾向得分估计完成！")
print(f"\n倾向得分摘要:")
print(df[df['day'] > 0][['day', 'ps_numerator', 'ps_denominator']].describe())

In [ ]:
# Step 2: 计算稳定化权重

def compute_stabilized_weights(df: pd.DataFrame) -> pd.DataFrame:
    """
    计算稳定化 IPTW 权重
    
    SW_i = ∏_t [P(A_t | A_bar_{t-1}) / P(A_t | A_bar_{t-1}, L_bar_t)]
    """
    df = df.copy()
    
    # 计算每个时点的权重比率
    df['weight_ratio'] = df['ps_numerator'] / df['ps_denominator']
    
    # 对每个用户，计算累积权重（乘积）
    df['sw'] = df.groupby('user_id')['weight_ratio'].cumprod()
    
    return df

df = compute_stabilized_weights(df)

print("✅ 权重计算完成！")
print(f"\n权重统计:")
final_weights = df[df['day'] == df['day'].max()]['sw']
print(f"  均值: {final_weights.mean():.3f}")
print(f"  中位数: {final_weights.median():.3f}")
print(f"  标准差: {final_weights.std():.3f}")
print(f"  最小值: {final_weights.min():.3f}")
print(f"  最大值: {final_weights.max():.3f}")
print(f"  99th percentile: {final_weights.quantile(0.99):.3f}")

In [ ]:
# 可视化权重分布

final_weights = df[df['day'] == df['day'].max()]['sw']

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=final_weights,
    nbinsx=50,
    marker_color=COLORS['primary'],
    opacity=0.7
))

fig.add_vline(x=1, line_dash="dash", line_color="red", annotation_text="理想值 = 1")

fig.update_layout(
    template='plotly_white',
    title='稳定化 IPTW 权重分布',
    xaxis_title='权重',
    yaxis_title='用户数',
    height=400
)

fig.show()

print("\n💡 解读：")
print("- 权重均值接近 1：稳定化成功")
print("- 如果有极端权重（>10）：可能需要截断")

In [ ]:
# 权重截断（可选，处理极端值）

def trim_weights(df: pd.DataFrame, percentile: float = 0.99) -> pd.DataFrame:
    """
    截断极端权重
    """
    df = df.copy()
    threshold = df['sw'].quantile(percentile)
    df['sw_trimmed'] = np.minimum(df['sw'], threshold)
    
    print(f"✅ 权重截断完成（99th percentile = {threshold:.3f}）")
    print(f"   影响 {(df['sw'] > threshold).sum()} 条记录")
    
    return df

df = trim_weights(df, percentile=0.99)

# 使用截断后的权重
df['sw_final'] = df['sw_trimmed']

In [ ]:
# Step 3: 拟合边际结构模型

def fit_marginal_structural_model(df: pd.DataFrame) -> Dict:
    """
    拟合边际结构模型
    
    模型：E[Y(a_bar)] = β_0 + β_1 * total_pushes
    
    使用加权最小二乘
    """
    # 提取每个用户的最终数据
    df_final = df.groupby('user_id').agg({
        'push': 'sum',  # 总推送次数
        'final_score': 'first',
        'sw_final': 'last'  # 最终权重
    }).reset_index()
    df_final.columns = ['user_id', 'total_pushes', 'final_score', 'weight']
    
    # 加权线性回归
    from sklearn.linear_model import LinearRegression
    
    X = df_final[['total_pushes']].values
    y = df_final['final_score'].values
    weights = df_final['weight'].values
    
    model = LinearRegression()
    model.fit(X, y, sample_weight=weights)
    
    beta_0 = model.intercept_
    beta_1 = model.coef_[0]
    
    print("✅ MSM 拟合完成！")
    print(f"\n边际结构模型:")
    print(f"  E[Y(a_bar)] = {beta_0:.2f} + {beta_1:.2f} × total_pushes")
    print(f"\n解释:")
    print(f"  - 每增加 1 次推送，平均成绩提升 {beta_1:.2f} 分")
    print(f"  - 完全不推送：预期成绩 {beta_0:.2f} 分")
    print(f"  - 推送 7 次：预期成绩 {beta_0 + 7 * beta_1:.2f} 分")
    print(f"  - 总效应：{7 * beta_1:.2f} 分")
    
    return {
        'model': model,
        'beta_0': beta_0,
        'beta_1': beta_1,
        'data': df_final
    }

msm_results = fit_marginal_structural_model(df)

In [ ]:
# 可视化 MSM 拟合结果

df_final = msm_results['data']
beta_0 = msm_results['beta_0']
beta_1 = msm_results['beta_1']

fig = go.Figure()

# 散点图（原始数据）
fig.add_trace(go.Scatter(
    x=df_final['total_pushes'],
    y=df_final['final_score'],
    mode='markers',
    marker=dict(
        size=df_final['weight'] * 5,  # 权重越大，点越大
        color=COLORS['neutral'],
        opacity=0.3
    ),
    name='观测数据',
    hovertemplate='推送次数: %{x}<br>成绩: %{y:.1f}<br>权重: %{marker.size:.2f}'
))

# MSM 拟合线
x_line = np.linspace(0, 7, 100)
y_line = beta_0 + beta_1 * x_line

fig.add_trace(go.Scatter(
    x=x_line,
    y=y_line,
    mode='lines',
    line=dict(color=COLORS['danger'], width=3),
    name=f'MSM: y = {beta_0:.1f} + {beta_1:.2f}x'
))

fig.update_layout(
    template='plotly_white',
    title='边际结构模型：推送次数的因果效应',
    xaxis_title='总推送次数',
    yaxis_title='最终成绩',
    height=500
)

fig.show()

## 🔬 对比：G-Computation

G-Computation 是另一种处理时变处理的方法，通过参数化建模每个时点的结果。

In [ ]:
# TODO 1: 实现 G-Computation
# 提示：
# 1. 对每个时点 t，建模 E[L_{t+1} | A_t, L_t]
# 2. 设定干预方案（如：所有人都推送）
# 3. 递归模拟干预后的结果
# 4. 平均得到 ATE

def g_computation(df: pd.DataFrame, intervention: str = 'always') -> float:
    """
    使用 G-Computation 估计因果效应
    
    参数:
        df: 数据
        intervention: 'always' (总是推送) 或 'never' (从不推送)
    
    返回:
        预期结果
    """
    # === 你的代码 ===
    
    # Step 1: 建模 study_hours_{t+1} = f(push_t, study_hours_t)
    # 使用线性回归
    
    
    # Step 2: 建模 final_score = g(total_study_hours, baseline_motivation)
    
    
    # Step 3: 模拟干预
    # 对每个用户，设置 push = 1 (always) 或 0 (never)
    # 递归预测 study_hours
    
    
    # Step 4: 预测最终成绩
    
    
    # === 代码结束 ===
    
    pass

# 测试
# ate_gcomp_always = g_computation(df, intervention='always')
# ate_gcomp_never = g_computation(df, intervention='never')
# ate_gcomp = ate_gcomp_always - ate_gcomp_never
# print(f"\nG-Computation ATE: {ate_gcomp:.2f}")

In [ ]:
<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def g_computation(df: pd.DataFrame, intervention: str = 'always') -> float:
    # Step 1: 建模学习时长动态
    df_model = df[df['day'] > 0].copy()
    prev = df[df['day'] >= 0].groupby('user_id').shift(1)
    df_model['study_lag'] = prev['study_hours']
    df_model['push_lag'] = prev['push']
    df_model = df_model.dropna()
    
    X = df_model[['push', 'study_lag']]
    y = df_model['study_hours']
    study_model = LinearRegression().fit(X, y)
    
    # Step 2: 模拟干预
    users = df['user_id'].unique()
    simulated_scores = []
    
    for user in users:
        user_data = df[df['user_id'] == user].sort_values('day')
        baseline_motivation = user_data['baseline_motivation'].iloc[0]
        study_hours = [user_data[user_data['day'] == 0]['study_hours'].values[0]]
        
        for day in range(1, 7):
            push = 1 if intervention == 'always' else 0
            X_pred = np.array([[push, study_hours[-1]]])
            study_next = study_model.predict(X_pred)[0]
            study_hours.append(max(0, min(5, study_next)))
        
        total_study = sum(study_hours)
        final_score = 50 + 0.3 * baseline_motivation + 3 * total_study
        simulated_scores.append(final_score)
    
    return np.mean(simulated_scores)

ate_gcomp_always = g_computation(df, intervention='always')
ate_gcomp_never = g_computation(df, intervention='never')
ate_gcomp = ate_gcomp_always - ate_gcomp_never
print(f"G-Computation ATE: {ate_gcomp:.2f}")
```

</details>

## 📊 方法对比：朴素 vs MSM-IPTW vs G-Computation

In [ ]:
# 计算不同方法的估计

# 1. 朴素估计（有偏）
user_summary = df.groupby('user_id').agg({
    'push': 'sum',
    'final_score': 'first'
}).reset_index()

high_push = user_summary[user_summary['push'] >= 5]['final_score'].mean()
low_push = user_summary[user_summary['push'] <= 2]['final_score'].mean()
naive_ate = high_push - low_push

# 2. MSM-IPTW
msm_ate = 7 * msm_results['beta_1']

# 3. 真实 ATE（如果有模拟数据的话）
# 这里我们假设真实 ATE ≈ MSM 估计

print("="*60)
print("方法对比：因果效应估计")
print("="*60)
print(f"\n1. 朴素估计（简单对比）")
print(f"   高频推送组成绩: {high_push:.2f}")
print(f"   低频推送组成绩: {low_push:.2f}")
print(f"   差异: {naive_ate:.2f} ⚠️ 有偏！")

print(f"\n2. MSM-IPTW")
print(f"   每次推送效应: {msm_results['beta_1']:.2f}")
print(f"   7次推送总效应: {msm_ate:.2f} ✅")

print(f"\n💡 关键发现：")
print(f"   朴素估计严重低估了推送的正向效应！")
print(f"   原因：学习表现差的用户更可能收到推送（选择偏差）")

In [ ]:
# 可视化对比

methods = ['朴素估计', 'MSM-IPTW']
estimates = [naive_ate, msm_ate]
colors_list = [COLORS['warning'], COLORS['success']]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=methods,
    y=estimates,
    marker_color=colors_list,
    text=[f"{e:.2f}" for e in estimates],
    textposition='outside'
))

fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig.update_layout(
    template='plotly_white',
    title='推送效应估计：不同方法对比',
    yaxis_title='7 次推送的总效应（成绩差异）',
    height=500
)

fig.show()

## 🔍 敏感性分析：权重截断的影响

In [ ]:
# TODO 2: 敏感性分析
# 提示：
# 1. 尝试不同的权重截断阈值（90th, 95th, 99th percentile）
# 2. 对每个阈值，重新拟合 MSM
# 3. 比较估计结果的稳定性

def sensitivity_to_trimming(df: pd.DataFrame, percentiles: List[float]) -> pd.DataFrame:
    """
    权重截断的敏感性分析
    """
    # === 你的代码 ===
    
    
    # === 代码结束 ===
    pass

# sensitivity_results = sensitivity_to_trimming(df, percentiles=[0.90, 0.95, 0.99, 1.00])
# print(sensitivity_results)

In [ ]:
<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def sensitivity_to_trimming(df: pd.DataFrame, percentiles: List[float]) -> pd.DataFrame:
    results = []
    
    for p in percentiles:
        df_temp = df.copy()
        
        if p < 1.0:
            threshold = df_temp['sw'].quantile(p)
            df_temp['sw_final'] = np.minimum(df_temp['sw'], threshold)
        else:
            df_temp['sw_final'] = df_temp['sw']
        
        # 拟合 MSM
        df_final = df_temp.groupby('user_id').agg({
            'push': 'sum',
            'final_score': 'first',
            'sw_final': 'last'
        }).reset_index()
        
        X = df_final[['push']].values
        y = df_final['final_score'].values
        w = df_final['sw_final'].values
        
        model = LinearRegression()
        model.fit(X, y, sample_weight=w)
        
        results.append({
            'percentile': p,
            'beta_1': model.coef_[0],
            'ate': 7 * model.coef_[0],
            'max_weight': df_temp['sw_final'].max()
        })
    
    return pd.DataFrame(results)

sensitivity_results = sensitivity_to_trimming(df, percentiles=[0.90, 0.95, 0.99, 1.00])
print("\n敏感性分析：权重截断对估计的影响")
print(sensitivity_results)
```

</details>

## 💡 思考题

### 基础理解

**1. 什么是时间依赖性混淆？为什么传统的 IPW 方法无法处理？**

提示：思考历史学习时长的双重角色

---

**2. 序贯忽略假设与标准忽略假设有什么区别？**

提示：条件集不同

---

### 深入分析

**3. 稳定化权重的分子和分母各代表什么？为什么要稳定化？**

提示：极端权重的问题

---

**4. MSM-IPTW 和 G-Computation 的核心区别是什么？各有什么优缺点？**

提示：参数化 vs 非参数化

---

**5. 在我们的例子中，朴素估计为什么会低估推送的效应？**

提示：选择偏差的方向

---

### 实战应用

**6. 如果你是这个在线教育平台的产品经理，基于 MSM 的结果，你会如何调整推送策略？**

提示：考虑推送频率、个性化

---

**7. 在哪些场景下，G-Computation 比 MSM-IPTW 更合适？**

提示：样本量、模型复杂度

---

**8. 如果权重分布非常不稳定（有极端值），除了截断，还有什么办法？**

提示：双重鲁棒估计

---

## 📋 本章小结

### 核心概念

| 概念 | 定义 | 重要性 |
|------|------|--------|
| **时变处理** | 个体在多个时点接受处理 $\bar{A}_t$ | 现实中大多数干预都是动态的 |
| **时间依赖性混淆** | 协变量既影响未来处理又影响结果，且自身受过去处理影响 | 传统方法失效的根本原因 |
| **序贯忽略假设** | $Y(\bar{a}) \perp A_t \mid \bar{A}_{t-1}, \bar{L}_t$ | 因果识别的关键假设 |
| **边际结构模型 (MSM)** | 建模干预方案 $\bar{a}$ 对结果的边际效应 | 估计目标 |
| **稳定化 IPTW** | $SW = \prod \frac{P(A_t \mid \bar{A}_{t-1})}{P(A_t \mid \bar{A}_{t-1}, \bar{L}_t)}$ | 核心估计方法 |
| **G-Computation** | 参数化建模 + 模拟干预 | 替代方法 |

---

### 方法对比

| 方法 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| **朴素估计** | ✅ 简单直观 | ❌ 严重有偏 | 仅用于探索性分析 |
| **MSM-IPTW** | ✅ 非参数化<br>✅ 稳健 | ❌ 权重不稳定<br>❌ 需要大样本 | 处理机制简单，样本充足 |
| **G-Computation** | ✅ 效率高<br>✅ 可处理复杂时间动态 | ❌ 模型依赖<br>❌ 误设风险 | 有充分领域知识 |
| **双重鲁棒** | ✅ 结合两者优点<br>✅ 只需一个模型正确 | ❌ 实现复杂 | 高风险决策 |

---

### 实践要点

| 步骤 | 关键点 | 常见陷阱 |
|------|--------|----------|
| **1. 识别混淆** | 画因果图，识别时间依赖性混淆 | 遗漏重要协变量 |
| **2. 估计倾向得分** | 每个时点建立模型，包含历史信息 | 模型误设 |
| **3. 计算权重** | 使用稳定化权重，检查分布 | 忽略极端权重 |
| **4. 拟合 MSM** | 加权回归，选择合适的函数形式 | 过度参数化 |
| **5. 敏感性分析** | 测试不同截断阈值、模型设定 | 只报告一个估计 |

---

### 关键公式速查

**稳定化 IPTW 权重：**

$$SW_i = \prod_{t=0}^{T} \frac{P(A_{it} = a_{it} \mid \bar{A}_{i,t-1})}{P(A_{it} = a_{it} \mid \bar{A}_{i,t-1}, \bar{L}_{it})}$$

**边际结构模型：**

$$E[Y(\bar{a})] = g(\bar{a}; \beta)$$

**序贯忽略假设：**

$$Y(\bar{a}) \perp A_t \mid \bar{A}_{t-1} = \bar{a}_{t-1}, \bar{L}_t \quad \forall t$$

---

### 下一步学习

1. **双重鲁棒估计**：结合 IPW 和 outcome regression
2. **动态治疗方案 (Dynamic Treatment Regimes)**：优化个性化干预策略
3. **中介分析 (Mediation Analysis)**：分解时间上的因果路径
4. **工具变量 (IV) 在面板数据中的应用**

---

**恭喜你完成了时变处理效应的学习！** 🎉

时变因果推断是因果推断中最复杂但也最贴近现实的领域。你现在掌握了处理动态干预问题的核心工具！